### Machine Learning Models

#### Dataset Preprocessing

In [ ]:
pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 24.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

In [ ]:
# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=42)

#### XGB Regressor

In [ ]:


# Train a random forest regressor
rf = XGBRegressor(n_estimators=500)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred = rf.predict(X_test)
print('R-squared:', rf.score(X_test, y_test))

R-squared: 0.7189138673680986


In [ ]:

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')


Mean Squared Error: 0.85
Root Mean Squared Error: 0.92
Mean Absolute Error: 0.62
R2 Score: 0.72


#### Random Forest Regressor

In [ ]:
# Train a random forest regressor
rf = RandomForestRegressor(n_estimators=500)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred = rf.predict(X_test)
print('R-squared:', rf.score(X_test, y_test))

R-squared: 0.7306572511229805


In [ ]:

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')


Mean Squared Error: 0.82
Root Mean Squared Error: 0.90
Mean Absolute Error: 0.61
R2 Score: 0.73


#### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

# Train a linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluate the model
y_pred = lr.predict(X_test)
print('R-squared:', lr.score(X_test, y_test))

R-squared: 0.4323337937221079


In [ ]:

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')


Mean Squared Error: 1.72
Root Mean Squared Error: 1.31
Mean Absolute Error: 0.93
R2 Score: 0.43


#### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=500)
gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

print('R-squared:', gb.score(X_test, y_test))

R-squared: 0.666936130727773


In [ ]:

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')


Mean Squared Error: 1.01
Root Mean Squared Error: 1.00
Mean Absolute Error: 0.76
R2 Score: 0.67


#### SVM

In [ ]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf', C=1.0)
svr.fit(X_train, y_train)

y_pred = svr.predict(X_test)

print('R-squared:', svr.score(X_test, y_test))

R-squared: 0.6851554012565448


In [ ]:

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')


Mean Squared Error: 0.95
Root Mean Squared Error: 0.98
Mean Absolute Error: 0.65
R2 Score: 0.69


### Deep Learning Models/Arhitectures/*Layers*

#### Conv1D Layer

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
np.random.seed(7)
tf.random.set_seed(7)

# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')
epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])


# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])


In [ ]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Conv1D, MaxPooling1D, Flatten

model = Sequential()
# No need to reshape here as X_train is already in the correct shape for Conv1D
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
# Remove the erroneous Reshape layer
model.add(Conv1D(64, 3, activation='relu', padding='same'))
# Adjust pool_size or remove MaxPooling1D if not needed
model.add(MaxPooling1D(pool_size=1))  # Change pool_size to 1
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['r2_score'])

# Fit the model on the training data
model.fit(X_train, y_train, epochs=250, batch_size=32, validation_data=(X_val, y_val), verbose=1)

Epoch 1/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 7.1862 - r2_score: -1.1982 - val_loss: 2.6481 - val_r2_score: 0.2043
Epoch 2/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 1.6368 - r2_score: 0.4965 - val_loss: 1.6806 - val_r2_score: 0.4950
Epoch 3/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.1084 - r2_score: 0.6578 - val_loss: 1.1450 - val_r2_score: 0.6560
Epoch 4/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.9648 - r2_score: 0.7024 - val_loss: 1.2130 - val_r2_score: 0.6355
Epoch 5/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.8212 - r2_score: 0.7466 - val_loss: 1.1169 - val_r2_score: 0.6644
Epoch 6/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.7467 - r2_score: 0.7698 - val_loss: 1.1380 - val_r2_score: 0.6581
Epoch 7/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.7040 - r2_score: 0.7830 - val_loss: 1.1227 - val_r2_score: 0.6627
Epoch 8/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.6938 - r2_score: 0.7863 - va

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Mean Squared Error: 1.16
Root Mean Squared Error: 1.08
Mean Absolute Error: 0.74
R2 Score: 0.63


In [ ]:
# Fit the model on the training data
model.fit(X_train, y_train, epochs=250, batch_size=32, validation_data=(X_val, y_val), verbose=1)

Epoch 1/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.2997 - r2_score: 0.9081 - val_loss: 1.1274 - val_r2_score: 0.6613
Epoch 2/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.3088 - r2_score: 0.9052 - val_loss: 1.2254 - val_r2_score: 0.6318
Epoch 3/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.2992 - r2_score: 0.9082 - val_loss: 1.2000 - val_r2_score: 0.6394
Epoch 4/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.3040 - r2_score: 0.9068 - val_loss: 1.2130 - val_r2_score: 0.6355
Epoch 5/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.2984 - r2_score: 0.9084 - val_loss: 1.1795 - val_r2_score: 0.6456
Epoch 6/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.2963 - r2_score: 0.9090 - val_loss: 1.1924 - val_r2_score: 0.6417
Epoch 7/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.2939 - r2_score: 0.9098 - val_loss: 1.1610 - val_r2_score: 0.6512
Epoch 8/250
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.3019 - r2_score: 0.9074 - val_l

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Mean Squared Error: 1.22
Root Mean Squared Error: 1.11
Mean Absolute Error: 0.75
R2 Score: 0.61


#### RNN

##### LSTM

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
np.random.seed(7)
tf.random.set_seed(7)

# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')
epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])


# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=42)



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np


import numpy as np
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)


In [ ]:
# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Initialize the model
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(64, return_sequences=True))  # Add first LSTM layer with return_sequences=True
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(LSTM(32, return_sequences=False))  # Add second LSTM layer with return_sequences=False
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(16, activation='tanh'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['r2_score'])

# Fit the model on the training data
model.fit(X_train, y_train, epochs=100,
          batch_size=32, validation_data=(X_val, y_val),
          verbose=1)


Epoch 1/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - loss: 8.1151 - r2_score: -1.4891 - val_loss: 2.7455 - val_r2_score: 0.1751
Epoch 2/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 2.7660 - r2_score: 0.1488 - val_loss: 2.0503 - val_r2_score: 0.3840
Epoch 3/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 1.9537 - r2_score: 0.3971 - val_loss: 1.5543 - val_r2_score: 0.5330
Epoch 4/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 1.6680 - r2_score: 0.4854 - val_loss: 1.3245 - val_r2_score: 0.6020
Epoch 5/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 1.5877 - r2_score: 0.5110 - val_loss: 1.3165 - val_r2_score: 0.6044
Epoch 6/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 1.4003 - r2_score: 0.5681 - val_loss: 1.2621 - val_r2_score: 0.6208
Epoch 7/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 1.2886 - r2_score: 0.6024 - val_loss: 1.2031 - val_r2_score: 0.6385
Epoch 8/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.2404 - r2_score: 0.6173 

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')

57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step
Mean Squared Error: 0.97
Root Mean Squared Error: 0.99
Mean Absolute Error: 0.65
R2 Score: 0.69


##### GRU

In [ ]:
pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 48.9 MB/s eta 0:00:00


In [ ]:

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
np.random.seed(7)
tf.random.set_seed(7)

# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')
epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])


# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=42)



In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)


In [ ]:
# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Initialize the model
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2]))) # Change the input shape to reflect the reshaped data
model.add(GRU(64, return_sequences=True))  # Add first LSTM layer with return_sequences=True
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(GRU(32, return_sequences=False))  # Add second LSTM layer with return_sequences=False
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(16, activation='tanh'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['r2_score'])

# Fit the model on the training data
model.fit(X_train, y_train, epochs=100,
          batch_size=32, validation_data=(X_val, y_val),
          verbose=1)

Epoch 1/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - loss: 8.2769 - r2_score: -1.5373 - val_loss: 1.9604 - val_r2_score: 0.4110
Epoch 2/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 2.6834 - r2_score: 0.1731 - val_loss: 1.7457 - val_r2_score: 0.4755
Epoch 3/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 2.0058 - r2_score: 0.3812 - val_loss: 1.4851 - val_r2_score: 0.5538
Epoch 4/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 1.6652 - r2_score: 0.4858 - val_loss: 1.4334 - val_r2_score: 0.5693
Epoch 5/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 1.5584 - r2_score: 0.5195 - val_loss: 1.3424 - val_r2_score: 0.5966
Epoch 6/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 1.3969 - r2_score: 0.5694 - val_loss: 1.3166 - val_r2_score: 0.6044
Epoch 7/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.2984 - r2_score: 0.5991 - val_loss: 1.2784 - val_r2_score: 0.6159
Epoch 8/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 1.2538 - r2_score: 0.6129 -

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Mean Squared Error: 0.98
Root Mean Squared Error: 0.99
Mean Absolute Error: 0.66
R2 Score: 0.68


##### BiLSTM

In [ ]:

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
np.random.seed(7)
tf.random.set_seed(7)

# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')
epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])


# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=42)



In [ ]:



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)


In [ ]:
# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Initialize the model
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2]))) # Change the input shape to reflect the reshaped data
model.add(Bidirectional(LSTM(64, return_sequences=True)))  # Add first BiLSTM layer with return_sequences=True
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(32)))  # Add second LSTM layer with return_sequences=False
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(16, activation='tanh'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['r2_score'])

# Fit the model on the training data
model.fit(X_train, y_train, epochs=100,
          batch_size=32, validation_data=(X_val, y_val),
          verbose=1)

Epoch 1/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - loss: 8.0092 - r2_score: -1.4549 - val_loss: 2.8009 - val_r2_score: 0.1584
Epoch 2/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 2.4605 - r2_score: 0.2422 - val_loss: 2.2359 - val_r2_score: 0.3282
Epoch 3/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 1.8673 - r2_score: 0.4249 - val_loss: 1.4644 - val_r2_score: 0.5600
Epoch 4/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 1.5979 - r2_score: 0.5064 - val_loss: 1.3103 - val_r2_score: 0.6063
Epoch 5/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 1.4263 - r2_score: 0.5601 - val_loss: 1.2368 - val_r2_score: 0.6284
Epoch 6/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 1.3165 - r2_score: 0.5939 - val_loss: 1.2098 - val_r2_score: 0.6365
Epoch 7/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 1.2079 - r2_score: 0.6271 - val_loss: 1.1840 - val_r2_score: 0.6442
Epoch 8/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 1.1911 - r2_score: 0.6325 

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')

57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step
Mean Squared Error: 0.95
Root Mean Squared Error: 0.97
Mean Absolute Error: 0.61
R2 Score: 0.70


##### BiGRU

In [ ]:

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
np.random.seed(7)
tf.random.set_seed(7)

# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')
epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])


# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)


In [ ]:
# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Initialize the model
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2]))) # Change the input shape to reflect the reshaped data
model.add(Bidirectional(GRU(64, return_sequences=True)))  # Add first BiGRU layer with return_sequences=True
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(GRU(32)))  # Add second LSTM layer with return_sequences=False
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(16, activation='tanh'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['r2_score'])

# Fit the model on the training data
model.fit(X_train, y_train, epochs=100,
          batch_size=32, validation_data=(X_val, y_val),
          verbose=1)

Epoch 1/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - loss: 7.8207 - r2_score: -1.3988 - val_loss: 1.9491 - val_r2_score: 0.4143
Epoch 2/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 2.3675 - r2_score: 0.2705 - val_loss: 1.7072 - val_r2_score: 0.4871
Epoch 3/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 1.9377 - r2_score: 0.4023 - val_loss: 1.4643 - val_r2_score: 0.5600
Epoch 4/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1.6299 - r2_score: 0.4969 - val_loss: 1.4265 - val_r2_score: 0.5714
Epoch 5/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 1.4352 - r2_score: 0.5569 - val_loss: 1.3273 - val_r2_score: 0.6012
Epoch 6/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 1.3301 - r2_score: 0.5892 - val_loss: 1.3333 - val_r2_score: 0.5994
Epoch 7/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 1.2690 - r2_score: 0.6084 - val_loss: 1.3814 - val_r2_score: 0.5849
Epoch 8/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 1.1614 - r2_score: 0.6417 

##### ConvLSTM1D

In [ ]:

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GRU, ConvLSTM1D
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
np.random.seed(7)
tf.random.set_seed(7)

# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')
epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])


# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Initialize the model
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2]))) # Change the input shape to reflect the reshaped data
#The input data needs to be reshaped to be 4 dimensional
model.add(tf.keras.layers.Reshape((1, 1, X_train.shape[2]))) # Reshape to (samples, time steps, rows, channels)
model.add(ConvLSTM1D(64, kernel_size=1, return_sequences=True))  # Add first LSTM layer with return_sequences=True
model.add(Dropout(0.5))
model.add(BatchNormalization())
# Remove the second reshape layer as it's not needed
model.add(ConvLSTM1D(32, kernel_size=1))  # Add second LSTM layer with return_sequences=False
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(16, activation='tanh'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['r2_score'])

# Fit the model on the training data
model.fit(X_train, y_train, epochs=100,
          batch_size=32, validation_data=(X_val, y_val),
          verbose=1)

Epoch 1/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 17s 34ms/step - loss: 8.2307 - r2_score: -1.5238 - val_loss: 2.6673 - val_r2_score: 0.1986
Epoch 2/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 2.6379 - r2_score: 0.1868 - val_loss: 2.0468 - val_r2_score: 0.3850
Epoch 3/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 1.9828 - r2_score: 0.3890 - val_loss: 1.5642 - val_r2_score: 0.5300
Epoch 4/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 1.6430 - r2_score: 0.4929 - val_loss: 1.4385 - val_r2_score: 0.5678
Epoch 5/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1.4928 - r2_score: 0.5390 - val_loss: 1.3297 - val_r2_score: 0.6005
Epoch 6/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 1.3600 - r2_score: 0.5804 - val_loss: 1.3226 - val_r2_score: 0.6026
Epoch 7/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 1.2851 - r2_score: 0.6035 - val_loss: 1.2563 - val_r2_score: 0.6225
Epoch 8/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1.2007 - r2_score: 0.6292 

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Mean Squared Error: 1.01
Root Mean Squared Error: 1.00
Mean Absolute Error: 0.67
R2 Score: 0.68


##### BiConvLSTM1D

In [ ]:

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GRU, ConvLSTM1D
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
np.random.seed(7)
tf.random.set_seed(7)

# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')
epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])


# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Initialize the model
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2]))) # Change the input shape to reflect the reshaped data
#The input data needs to be reshaped to be 4 dimensional
model.add(tf.keras.layers.Reshape((1, 1, X_train.shape[2]))) # Reshape to (samples, time steps, rows, channels)
model.add(Bidirectional(ConvLSTM1D(64, kernel_size=1, return_sequences=True)))  # Add first LSTM layer with return_sequences=True
model.add(Dropout(0.5))
model.add(BatchNormalization())
# Remove the second reshape layer as it's not needed
model.add(Bidirectional(ConvLSTM1D(32, kernel_size=1)))  # Add second LSTM layer with return_sequences=False
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(16, activation='tanh'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['r2_score'])

# Fit the model on the training data
model.fit(X_train, y_train, epochs=100,
          batch_size=32, validation_data=(X_val, y_val),
          verbose=1)

Epoch 1/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 27s 58ms/step - loss: 7.8306 - r2_score: -1.3987 - val_loss: 2.9380 - val_r2_score: 0.1172
Epoch 2/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 2.4580 - r2_score: 0.2425 - val_loss: 2.0579 - val_r2_score: 0.3817
Epoch 3/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - loss: 1.8279 - r2_score: 0.4361 - val_loss: 1.5238 - val_r2_score: 0.5422
Epoch 4/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 1.6000 - r2_score: 0.5064 - val_loss: 1.3754 - val_r2_score: 0.5867
Epoch 5/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - loss: 1.4759 - r2_score: 0.5445 - val_loss: 1.3166 - val_r2_score: 0.6044
Epoch 6/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 12s 45ms/step - loss: 1.3182 - r2_score: 0.5934 - val_loss: 1.2335 - val_r2_score: 0.6294
Epoch 7/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - loss: 1.2437 - r2_score: 0.6160 - val_loss: 1.2781 - val_r2_score: 0.6160
Epoch 8/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - loss: 1.1768 - r2_score: 0.63

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score: {r2:.2f}')

57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step
Mean Squared Error: 0.95
Root Mean Squared Error: 0.97
Mean Absolute Error: 0.61
R2 Score: 0.70


#### Encoder

###### VAE

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)

# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = AllChem.GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])

# Extract the labels
y = df['pIC50'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the VAE model
class VariationalAutoencoder(tf.keras.Model):
    def __init__(self, latent_dim, input_dim):
        super(VariationalAutoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.input_dim = input_dim

        # Encoder network
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(latent_dim + latent_dim),  # Mean and variance of the latent space
        ])

        # Decoder network
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(latent_dim,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim, activation='sigmoid'),  # Output layer with sigmoid activation for binary data
        ])

    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        epsilon = tf.random.normal(shape=mean.shape)
        return mean + tf.exp(0.5 * logvar) * epsilon

    def decode(self, z):
        return self.decoder(z)

    def call(self, x):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        reconstructed = self.decode(z)
        return reconstructed

# Define the regression model
class RegressionModel(tf.keras.Model):
    def __init__(self, latent_dim):
        super(RegressionModel, self).__init__()
        self.regression_network = tf.keras.Sequential([
            layers.InputLayer(input_shape=(latent_dim,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(1),  # Output layer with 1 unit for continuous target variable
        ])

    def call(self, z):
        return self.regression_network(z)

# Define the loss functions
def vae_loss(x, reconstructed, mean, logvar):
    # Reconstruction loss (binary cross-entropy)
    reconstruction_loss = tf.reduce_mean(
        tf.keras.losses.binary_crossentropy(x, reconstructed)
    )
    # KL divergence loss
    kl_loss = -0.5 * tf.reduce_mean(1 + logvar - tf.square(mean) - tf.exp(logvar))
    return reconstruction_loss + kl_loss

def regression_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float64)
    y_pred = tf.cast(y_pred, dtype=tf.float64)
    return tf.reduce_mean(tf.square(y_true - y_pred))

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        reconstructed = vae(x)
        mean, logvar = vae.encode(x)
        z = vae.reparameterize(mean, logvar)
        y_pred = regression_model(z)
        vae_loss_value = vae_loss(x, reconstructed, mean, logvar)
        regression_loss_value = regression_loss(y, y_pred)
        vae_loss_value = tf.cast(vae_loss_value, dtype=tf.float32)
        # Cast regression_loss_value to float32 to match vae_loss_value
        regression_loss_value = tf.cast(regression_loss_value, dtype=tf.float32)
        total_loss = vae_loss_value + regression_loss_value
    gradients = tape.gradient(total_loss, vae.trainable_variables + regression_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, vae.trainable_variables + regression_model.trainable_variables))
    return vae_loss_value, regression_loss_value

# Define the model and optimizer
latent_dim = 16
input_dim = X.shape[1]
vae = VariationalAutoencoder(latent_dim, input_dim)
regression_model = RegressionModel(latent_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    vae_loss_value, regression_loss_value = train_step(X_train, y_train)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: VAE Loss = {vae_loss_value:.4f}, Regression Loss = {regression_loss_value:.4f}")

# Evaluate the model on the test set
def evaluate_model(x, y):
    reconstructed = vae(x)
    mean, logvar = vae.encode(x)
    z = vae.reparameterize(mean, logvar)
    y_pred = regression_model(z)
    vae_loss_value = vae_loss(x, reconstructed, mean, logvar)
    regression_loss_value = regression_loss(y, y_pred)
    r2 = r2_score(y, y_pred)
    return vae_loss_value, regression_loss_value, r2

# Evaluate and print the results on the test set
vae_loss_test, regression_loss_test, r2_test = evaluate_model(X_test, y_test)
print(f"Test VAE Loss = {vae_loss_test:.4f}, Test Regression Loss = {regression_loss_test:.4f}, Test R² Score = {r2_test:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 0: VAE Loss = 0.7091, Regression Loss = 11.5092
Epoch 10: VAE Loss = 1.7567, Regression Loss = 4.2519
Epoch 20: VAE Loss = 0.4550, Regression Loss = 3.9510
Epoch 30: VAE Loss = 0.4313, Regression Loss = 3.5829
Epoch 40: VAE Loss = 0.3278, Regression Loss = 3.5604
Epoch 50: VAE Loss = 0.3015, Regression Loss = 3.5186
Epoch 60: VAE Loss = 0.2897, Regression Loss = 3.4747
Epoch 70: VAE Loss = 0.2554, Regression Loss = 3.4724
Epoch 80: VAE Loss = 0.2394, Regression Loss = 3.4557
Epoch 90: VAE Loss = 0.2163, Regression Loss = 3.4529
Test VAE Loss = 0.1990, Test Regression Loss = 3.2145, Test R² Score = -0.0712


## Bayesain Optimization & Cross Validation

In [ ]:
pip install scikit-optimize

In [ ]:
import pandas as pd
import numpy as np
from skopt import BayesSearchCV
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

In [ ]:
# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=42)

##### XGBOOST with Bayesain Optimization and Cross Validation

In [ ]:
from xgboost import XGBRegressor

# Initialize the model
xgb_model = XGBRegressor()
# Define the hyperparameter space to search
param_space = {
    'n_estimators': (100, 500, 1000),               # Number of trees
    'max_depth': (3, 10),                    # Maximum tree depth
    'learning_rate': (0.01, 0.3, 'log-uniform'),  # Learning rate
    'subsample': (0.6, 1.0),                 # Subsample ratio of the training instances
    'colsample_bytree': (0.6, 1.0),          # Subsample ratio of columns when constructing each tree
    'gamma': (0, 1),                         # Minimum loss reduction required to make a further partition on a leaf node
}

# Bayesian Optimization
opt = BayesSearchCV(
    estimator=xgb_model,
    search_spaces=param_space,
    n_iter=3,  # Number of parameter settings to try (iterations)
    cv=5,  # Cross-validation splitting strategy
    n_jobs=-1,  # Use all available cores
    random_state=7
)

# Perform the search
opt.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best Parameters: {opt.best_params_}")
print(f"Best CV Score: {opt.best_score_}")

# Evaluate on test set
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate and print metrics
from sklearn.metrics import mean_squared_error, r2_score

print(f"Test R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")

Best Parameters: OrderedDict([('colsample_bytree', 0.8302362481204948), ('gamma', 1), ('learning_rate', 0.09050237944295461), ('max_depth', 7), ('n_estimators', 500), ('subsample', 0.756376101203944)])
Best CV Score: 0.7087705188867961
Test R2 Score: 0.7274
Test MSE: 0.8251
Test MAE: 0.6335
Test RMSE: 0.9083


##### Random Forest Regressor with Bayesain Optimization and Cross Validation

In [ ]:
import numpy as np
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

# Assuming you have your dataset loaded into X and y
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
rf_model = RandomForestRegressor()

# Define the hyperparameter space to search
param_space = {
    'n_estimators': (100, 500, 1000),               # Number of trees
    'max_depth': (3, 10),                            # Maximum tree depth
    'min_samples_split': (2, 10),                    # Minimum number of samples required to split an internal node
    'min_samples_leaf': (1, 5),                      # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],         # Number of features to consider when looking for the best split
    'bootstrap': [True, False]                        # Whether bootstrap samples are used when building trees
}

# Bayesian Optimization
opt = BayesSearchCV(
    estimator=rf_model,
    search_spaces=param_space,
    n_iter=3,  # Number of parameter settings to try (iterations)
    cv=5,  # Cross-validation splitting strategy
    n_jobs=-1,  # Use all available cores
    random_state=7
)

# Perform the search
opt.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best Parameters: {opt.best_params_}")
print(f"Best CV Score: {opt.best_score_}")

# Evaluate on test set
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate and print metrics
print(f"Test R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")


Best Parameters: OrderedDict([('bootstrap', False), ('max_depth', 10), ('max_features', 'sqrt'), ('min_samples_leaf', 3), ('min_samples_split', 5), ('n_estimators', 500)])
Best CV Score: 0.4808880107182902
Test R2 Score: 0.4749
Test MSE: 1.5893
Test MAE: 0.9877
Test RMSE: 1.2607


##### Linear Regression with Bayesain Optimization and Cross Validation

In [ ]:
import numpy as np
from skopt import BayesSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Assuming you have your dataset loaded into X and y
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model with a pipeline that includes StandardScaler
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Define the hyperparameter space to search
param_space = {
    'regressor__fit_intercept': [True, False],  # Whether to calculate the intercept for this model
}

# Bayesian Optimization
opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_space,
    n_iter=3,  # Number of parameter settings to try (iterations)
    cv=5,  # Cross-validation splitting strategy
    n_jobs=-1,  # Use all available cores
    random_state=7
)

# Perform the search
opt.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best Parameters: {opt.best_params_}")
print(f"Best CV Score: {opt.best_score_}")

# Evaluate on test set
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate and print metrics
print(f"Test R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")


Best Parameters: OrderedDict([('regressor__fit_intercept', True)])
Best CV Score: -1.2492579941164161e+21
Test R2 Score: 0.4323
Test MSE: 1.7182
Test MAE: 0.9266
Test RMSE: 1.3108


##### Gradient Boosting Regressor with Bayesain Optimization and Cross Validation

In [ ]:
import numpy as np
from skopt import BayesSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

# Assuming you have your dataset loaded into X and y
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
gb_model = GradientBoostingRegressor()

# Define the hyperparameter space to search
param_space = {
    'n_estimators': (100, 500, 1000),               # Number of boosting stages
    'learning_rate': (0.01, 0.3, 'log-uniform'),    # Learning rate
    'max_depth': (3, 10),                           # Maximum depth of the individual regression estimators
    'min_samples_split': (2, 10),                   # Minimum number of samples required to split an internal node
    'min_samples_leaf': (1, 5),                     # Minimum number of samples required to be at a leaf node
    'subsample': (0.6, 1.0),                        # Fraction of samples used to fit the individual base learners
    'max_features': ['auto', 'sqrt', 'log2'],       # Number of features to consider when looking for the best split
}

# Bayesian Optimization
opt = BayesSearchCV(
    estimator=gb_model,
    search_spaces=param_space,
    n_iter=3,  # Number of parameter settings to try (iterations)
    cv=5,  # Cross-validation splitting strategy
    n_jobs=-1,  # Use all available cores
    random_state=7
)

# Perform the search
opt.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best Parameters: {opt.best_params_}")
print(f"Best CV Score: {opt.best_score_}")

# Evaluate on test set
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate and print metrics
print(f"Test R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")


Best Parameters: OrderedDict([('learning_rate', 0.07082998327749068), ('max_depth', 10), ('max_features', 'sqrt'), ('min_samples_leaf', 3), ('min_samples_split', 5), ('n_estimators', 500), ('subsample', 0.8837231239399538)])
Best CV Score: 0.7156947966516782
Test R2 Score: 0.7340
Test MSE: 0.8052
Test MAE: 0.6357
Test RMSE: 0.8973


##### SVR with Bayesain Optimization and Cross Validation

In [ ]:
import numpy as np
from skopt import BayesSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Assuming you have your dataset loaded into X and y
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model with a pipeline that includes StandardScaler
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', SVR())
])

# Define the hyperparameter space to search
param_space = {
    'regressor__C': (0.1, 10, 'log-uniform'),  # Regularization parameter
    'regressor__epsilon': (0.01, 1.0, 'log-uniform'),  # Epsilon in the epsilon-SVR model
    'regressor__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Specifies the kernel type to be used in the algorithm
    'regressor__degree': (2, 5),  # Degree of the polynomial kernel function ('poly')
    'regressor__gamma': (0.001, 1.0, 'log-uniform'),  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    'regressor__coef0': (0.0, 1.0)  # Independent term in kernel function. It is only significant in 'poly' and 'sigmoid'
}

# Bayesian Optimization
opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_space,
    n_iter=4,  # Number of parameter settings to try (iterations)
    cv=5,  # Cross-validation splitting strategy
    n_jobs=-1,  # Use all available cores
    random_state=7
)

# Perform the search
opt.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best Parameters: {opt.best_params_}")
print(f"Best CV Score: {opt.best_score_}")

# Evaluate on test set
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate and print metrics
print(f"Test R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")


Best Parameters: OrderedDict([('regressor__C', 1.4163847450235516), ('regressor__coef0', 0.9879356418147209), ('regressor__degree', 4), ('regressor__epsilon', 0.1263929516246042), ('regressor__gamma', 0.016540213837160592), ('regressor__kernel', 'poly')])
Best CV Score: 0.6425594996917909
Test R2 Score: 0.6639
Test MSE: 1.0174
Test MAE: 0.6772
Test RMSE: 1.0087


In [ ]:
pip install rdkit scikit-optimize


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 4.7 MB/s eta 0:00:00


Convo1D

##### BiLSTM with Bayesain optimization and Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Global variable to store the best model
best_model = None
best_optimizer = None

# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(32, activation='relu')))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

def objective_function(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    global best_optimizer

    # Round the optimizer index to the nearest integer
    optimizer_idx = round(optimizer_idx)

    model = create_model(optimizer_idx, learning_rate, dropout_rate, batch_size)

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    history = model.fit(X_train, y_train, epochs=100, batch_size=int(batch_size), validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Calculate the validation score
    y_val_pred = model.predict(X_val)
    val_score = r2_score(y_val, y_val_pred)

    # Update the best model if necessary
    if best_model is None or val_score > np.mean(r2_score(y_val, best_model.predict(X_val))):
        best_model = model
        best_optimizer = optimizers[optimizer_idx]

    # Return the validation score
    return val_score #last will be return

# Define the parameter space for Bayesian optimization
pbounds = {
    'optimizer_idx': (0, len(optimizers) - 1),
    'learning_rate': (0.0001, 0.1),
    'dropout_rate': (0.1, 0.5),
    'batch_size': (16, 64)
}

# Create the Bayesian optimization object
optimizer = BayesianOptimization(f=objective_function, pbounds=pbounds, random_state=7)

# Run the Bayesian optimization
# optimizer.maximize(init_points=5, n_iter=3)



In [ ]:
# Optimize the model
optimizer.maximize(init_points=5, n_iter=3)  # You can adjust the number of iterations

|   iter    |  target   | batch_... | dropou... | learni... | optimi... |
-------------------------------------------------------------------------
Epoch 1/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - loss: 3.4923 - r2_score: -0.1056 - val_loss: 4.1317 - val_r2_score: -0.2414
Epoch 2/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.1816 - r2_score: 0.3163 - val_loss: 2.9565 - val_r2_score: 0.1117
Epoch 3/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 1.8620 - r2_score: 0.4241 - val_loss: 1.4608 - val_r2_score: 0.5611
Epoch 4/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 1.9106 - r2_score: 0.4239 - val_loss: 1.7962 - val_r2_score: 0.4603
Epoch 5/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - loss: 1.6394 - r2_score: 0.4852 - val_loss: 1.5258 - val_r2_score: 0.5416
Epoch 6/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - loss: 1.6627 - r2_score: 0.4770 - val_loss: 1.4880 - val_r2_score: 0.5529
Epoch 7/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 1.5160

In [ ]:

# Print the best parameters found by Bayesian Optimization
print('Best parameters:', optimizer.max)
print('Best optimizer:', best_optimizer)

# Train the model with the best parameters
best_params = optimizer.max['params']
best_optimizer_idx = best_params['optimizer_idx']
best_learning_rate = best_params['learning_rate']
best_dropout_rate = best_params['dropout_rate']
best_batch_size = best_params['batch_size']

# Use the best model stored in the global variable
model = best_model

# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score using optimized model: {r2:.2f}')


# Store the average R-squared score
avg_score = optimizer.max['target']
print(f'Average R2 Score: {avg_score:.2f}')


Best parameters: {'target': 0.6790141568738213, 'params': {'batch_size': 34.28517439112984, 'dropout_rate': 0.12637453876236204, 'learning_rate': 0.028885745370868555, 'optimizer_idx': 1.8191870554392273}}
Best optimizer: SGD
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Mean Squared Error: 1.10
Root Mean Squared Error: 1.05
Mean Absolute Error: 0.68
R2 Score using optimized model: 0.65
Average R2 Score: 0.68


BiLSTM Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(32, activation='relu')))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the number of folds
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

# Lists to store the scores for each fold
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []



In [ ]:
# Perform cross-validation
fold_number = 0
for train, val in kfold.split(X_train, y_train):
    fold_number += 1
    print(f"Starting fold {fold_number}")

    # Split the training data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train], X_train[val]

    # Use .iloc[] to access elements by integer position
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    # Create the model with the best parameters found by Bayesian optimization
    model = create_model(best_params['optimizer_idx'], best_params['learning_rate'], best_params['dropout_rate'], best_params['batch_size'])

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    # Train the model on the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=int(best_params['batch_size']), validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

    # Make predictions on the test data for this fold
    y_pred_fold = model.predict(X_test).flatten()

    # Calculate the mean squared error, mean absolute error, root mean squared error, and R-squared score for this fold
    mse = mean_squared_error(y_test, y_pred_fold)
    mae = mean_absolute_error(y_test, y_pred_fold)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_fold)

    # Append the scores to the lists of scores
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)


Starting fold 1
Epoch 1/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - loss: 3.2963 - r2_score: -0.0398 - val_loss: 1.9439 - val_r2_score: 0.4097
Epoch 2/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 1.5049 - r2_score: 0.5402 - val_loss: 2.2147 - val_r2_score: 0.3274
Epoch 3/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 1.2526 - r2_score: 0.6231 - val_loss: 1.5479 - val_r2_score: 0.5299
Epoch 4/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1.0394 - r2_score: 0.6742 - val_loss: 1.1586 - val_r2_score: 0.6481
Epoch 5/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.9190 - r2_score: 0.7030 - val_loss: 1.5249 - val_r2_score: 0.5369
Epoch 6/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.9210 - r2_score: 0.7141 - val_loss: 1.4890 - val_r2_score: 0.5478
Epoch 7/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 0.8838 - r2_score: 0.7210 - val_loss: 1.5594 - val_r2_score: 0.5264
Epoch 8/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.7974 - r

In [ ]:

print("Cross Validation of using best parameters")
# Print the average mean squared error, mean absolute error, root mean squared error, and R-squared score across all folds
print(f'Average mean squared error (5-fold cross-validation): {np.mean(mse_scores):.2f}')
print(f'Average mean absolute error (5-fold cross-validation): {np.mean(mae_scores):.2f}')
print(f'Average root mean squared error (5-fold cross-validation): {np.mean(rmse_scores):.2f}')
print(f'Average R-squared score (5-fold cross-validation): {np.mean(r2_scores):.2f}')


Cross Validation of using best parameters
Average mean squared error (5-fold cross-validation): 2.08
Average mean absolute error (5-fold cross-validation): 0.99
Average root mean squared error (5-fold cross-validation): 1.43
Average R-squared score (5-fold cross-validation): 0.33


##### BiGRU with Bayesain optimization and Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Global variable to store the best model
best_model = None
best_optimizer = None

# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Bidirectional(GRU(64, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Bidirectional(GRU(32, activation='relu')))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

def objective_function(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    global best_optimizer

    # Round the optimizer index to the nearest integer
    optimizer_idx = round(optimizer_idx)

    model = create_model(optimizer_idx, learning_rate, dropout_rate, batch_size)

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    history = model.fit(X_train, y_train, epochs=100, batch_size=int(batch_size), validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Calculate the validation score
    y_val_pred = model.predict(X_val)
    val_score = r2_score(y_val, y_val_pred)

    # Update the best model if necessary
    if best_model is None or val_score > np.mean(r2_score(y_val, best_model.predict(X_val))):
        best_model = model
        best_optimizer = optimizers[optimizer_idx]

    # Return the validation score
    return val_score #last will be return

# Define the parameter space for Bayesian optimization
pbounds = {
    'optimizer_idx': (0, len(optimizers) - 1),
    'learning_rate': (0.0001, 0.1),
    'dropout_rate': (0.1, 0.5),
    'batch_size': (16, 64)
}

# Create the Bayesian optimization object
optimizer = BayesianOptimization(f=objective_function, pbounds=pbounds, random_state=7)

# Run the Bayesian optimization
# optimizer.maximize(init_points=5, n_iter=3)



In [ ]:
# Optimize the model
optimizer.maximize(init_points=5, n_iter=3)  # You can adjust the number of iterations

|   iter    |  target   | batch_... | dropou... | learni... | optimi... |
-------------------------------------------------------------------------
Epoch 1/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - loss: 3.6502 - r2_score: -0.1605 - val_loss: 1.8013 - val_r2_score: 0.4588
Epoch 2/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 2.1330 - r2_score: 0.3275 - val_loss: 1.9901 - val_r2_score: 0.4021
Epoch 3/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 13s 31ms/step - loss: 1.8260 - r2_score: 0.4208 - val_loss: 3.0539 - val_r2_score: 0.0824
Epoch 4/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 1.7867 - r2_score: 0.4321 - val_loss: 1.4808 - val_r2_score: 0.5551
Epoch 5/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 1.6859 - r2_score: 0.4658 - val_loss: 1.6736 - val_r2_score: 0.4971
Epoch 6/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - loss: 1.6511 - r2_score: 0.4814 - val_loss: 2.3805 - val_r2_score: 0.2847
Epoch 7/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 1.5790 

In [ ]:

# Print the best parameters found by Bayesian Optimization
print('Best parameters:', optimizer.max)
print('Best optimizer:', best_optimizer)

# Train the model with the best parameters
best_params = optimizer.max['params']
best_optimizer_idx = best_params['optimizer_idx']
best_learning_rate = best_params['learning_rate']
best_dropout_rate = best_params['dropout_rate']
best_batch_size = best_params['batch_size']

# Use the best model stored in the global variable
model = best_model

# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score using optimized model: {r2:.2f}')


# Store the average R-squared score
avg_score = optimizer.max['target']
print(f'Average R2 Score: {avg_score:.2f}')


Best parameters: {'target': 0.7000594110017124, 'params': {'batch_size': 34.28517439112984, 'dropout_rate': 0.12637453876236204, 'learning_rate': 0.028885745370868555, 'optimizer_idx': 1.8191870554392273}}
Best optimizer: SGD
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Mean Squared Error: 1.04
Root Mean Squared Error: 1.02
Mean Absolute Error: 0.67
R2 Score using optimized model: 0.67
Average R2 Score: 0.70


BiGRU Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(32, activation='relu')))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the number of folds
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

# Lists to store the scores for each fold
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []



In [ ]:
# Perform cross-validation
fold_number = 0
for train, val in kfold.split(X_train, y_train):
    fold_number += 1
    print(f"Starting fold {fold_number}")

    # Split the training data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train], X_train[val]

    # Use .iloc[] to access elements by integer position
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    # Create the model with the best parameters found by Bayesian optimization
    model = create_model(best_params['optimizer_idx'], best_params['learning_rate'], best_params['dropout_rate'], best_params['batch_size'])

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    # Train the model on the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=int(best_params['batch_size']), validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

    # Make predictions on the test data for this fold
    y_pred_fold = model.predict(X_test).flatten()

    # Calculate the mean squared error, mean absolute error, root mean squared error, and R-squared score for this fold
    mse = mean_squared_error(y_test, y_pred_fold)
    mae = mean_absolute_error(y_test, y_pred_fold)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_fold)

    # Append the scores to the lists of scores
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)


Starting fold 1
Epoch 1/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 3.0923 - r2_score: 0.0685 - val_loss: 1.6555 - val_r2_score: 0.4973
Epoch 2/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1.6238 - r2_score: 0.4982 - val_loss: 1.7140 - val_r2_score: 0.4795
Epoch 3/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 1.2959 - r2_score: 0.5943 - val_loss: 1.7359 - val_r2_score: 0.4728
Epoch 4/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 1.1046 - r2_score: 0.6689 - val_loss: 2.4971 - val_r2_score: 0.2417
Epoch 5/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 1.0346 - r2_score: 0.6686 - val_loss: 1.2124 - val_r2_score: 0.6318
Epoch 6/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 0.8818 - r2_score: 0.7258 - val_loss: 1.9901 - val_r2_score: 0.3956
Epoch 7/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.9172 - r2_score: 0.7079 - val_loss: 1.4879 - val_r2_score: 0.5482
Epoch 8/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.8738 - r2

In [ ]:

print("Cross Validation of using best parameters")
# Print the average mean squared error, mean absolute error, root mean squared error, and R-squared score across all folds
print(f'Average mean squared error (5-fold cross-validation): {np.mean(mse_scores):.2f}')
print(f'Average mean absolute error (5-fold cross-validation): {np.mean(mae_scores):.2f}')
print(f'Average root mean squared error (5-fold cross-validation): {np.mean(rmse_scores):.2f}')
print(f'Average R-squared score (5-fold cross-validation): {np.mean(r2_scores):.2f}')


Cross Validation of using best parameters
Average mean squared error (5-fold cross-validation): 2.17
Average mean absolute error (5-fold cross-validation): 1.01
Average root mean squared error (5-fold cross-validation): 1.44
Average R-squared score (5-fold cross-validation): 0.30


##### LSTM with Bayesain optimization  and Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Global variable to store the best model
best_model = None
best_optimizer = None

# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(LSTM(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

def objective_function(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    global best_optimizer

    # Round the optimizer index to the nearest integer
    optimizer_idx = round(optimizer_idx)

    model = create_model(optimizer_idx, learning_rate, dropout_rate, batch_size)

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    history = model.fit(X_train, y_train, epochs=100, batch_size=int(batch_size), validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Calculate the validation score
    y_val_pred = model.predict(X_val)
    val_score = r2_score(y_val, y_val_pred)

    # Update the best model if necessary
    if best_model is None or val_score > np.mean(r2_score(y_val, best_model.predict(X_val))):
        best_model = model
        best_optimizer = optimizers[optimizer_idx]

    # Return the validation score
    return val_score #last will be return

# Define the parameter space for Bayesian optimization
pbounds = {
    'optimizer_idx': (0, len(optimizers) - 1),
    'learning_rate': (0.0001, 0.1),
    'dropout_rate': (0.1, 0.5),
    'batch_size': (16, 64)
}

# Create the Bayesian optimization object
optimizer = BayesianOptimization(f=objective_function, pbounds=pbounds, random_state=7)

# Run the Bayesian optimization
# optimizer.maximize(init_points=5, n_iter=3)



In [ ]:
# Optimize the model
optimizer.maximize(init_points=5, n_iter=3)  # You can adjust the number of iterations

|   iter    |  target   | batch_... | dropou... | learni... | optimi... |
-------------------------------------------------------------------------
Epoch 1/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - loss: 3.4259 - r2_score: -0.0402 - val_loss: 3.3668 - val_r2_score: -0.0116
Epoch 2/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 2.2651 - r2_score: 0.3203 - val_loss: 3.0296 - val_r2_score: 0.0897
Epoch 3/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 1.9115 - r2_score: 0.4022 - val_loss: 1.7977 - val_r2_score: 0.4599
Epoch 4/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - loss: 1.6804 - r2_score: 0.4627 - val_loss: 3.6598 - val_r2_score: -0.0997
Epoch 5/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - loss: 1.8421 - r2_score: 0.3912 - val_loss: 2.0581 - val_r2_score: 0.3816
Epoch 6/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - loss: 1.7488 - r2_score: 0.4676 - val_loss: 2.3804 - val_r2_score: 0.2848
Epoch 7/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 1.6443

In [ ]:

# Print the best parameters found by Bayesian Optimization
print('Best parameters:', optimizer.max)
print('Best optimizer:', best_optimizer)

# Train the model with the best parameters
best_params = optimizer.max['params']
best_optimizer_idx = best_params['optimizer_idx']
best_learning_rate = best_params['learning_rate']
best_dropout_rate = best_params['dropout_rate']
best_batch_size = best_params['batch_size']

# Use the best model stored in the global variable
model = best_model

# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score using optimized model: {r2:.2f}')


# Store the average R-squared score
avg_score = optimizer.max['target']
print(f'Average R2 Score: {avg_score:.2f}')


Best parameters: {'target': 0.6874101785804605, 'params': {'batch_size': 28.885071044889816, 'dropout_rate': 0.299953000330224, 'learning_rate': 0.06795507661248196, 'optimizer_idx': 1.607478072208751}}
Best optimizer: SGD
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Mean Squared Error: 1.05
Root Mean Squared Error: 1.02
Mean Absolute Error: 0.73
R2 Score using optimized model: 0.66
Average R2 Score: 0.69


LSTM Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(LSTM(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the number of folds
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

# Lists to store the scores for each fold
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []



In [ ]:
# Perform cross-validation
fold_number = 0
for train, val in kfold.split(X_train, y_train):
    fold_number += 1
    print(f"Starting fold {fold_number}")

    # Split the training data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train], X_train[val]

    # Use .iloc[] to access elements by integer position
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    # Create the model with the best parameters found by Bayesian optimization
    model = create_model(best_params['optimizer_idx'], best_params['learning_rate'], best_params['dropout_rate'], best_params['batch_size'])

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    # Train the model on the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=int(best_params['batch_size']), validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

    # Make predictions on the test data for this fold
    y_pred_fold = model.predict(X_test).flatten()

    # Calculate the mean squared error, mean absolute error, root mean squared error, and R-squared score for this fold
    mse = mean_squared_error(y_test, y_pred_fold)
    mae = mean_absolute_error(y_test, y_pred_fold)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_fold)

    # Append the scores to the lists of scores
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)


Starting fold 1
Epoch 1/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 3.5875 - r2_score: -0.0989 - val_loss: 3.2790 - val_r2_score: 0.0042
Epoch 2/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 2.2461 - r2_score: 0.3090 - val_loss: 1.7780 - val_r2_score: 0.4600
Epoch 3/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.8050 - r2_score: 0.4120 - val_loss: 2.5229 - val_r2_score: 0.2338
Epoch 4/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.8429 - r2_score: 0.4260 - val_loss: 1.3955 - val_r2_score: 0.5762
Epoch 5/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 1.6163 - r2_score: 0.5112 - val_loss: 1.5940 - val_r2_score: 0.5159
Epoch 6/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 1.6075 - r2_score: 0.4889 - val_loss: 1.8034 - val_r2_score: 0.4523
Epoch 7/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 1.5962 - r2_score: 0.5198 - val_loss: 2.0985 - val_r2_score: 0.3627
Epoch 8/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.5231 - r2

In [ ]:

print("Cross Validation of using best parameters")
# Print the average mean squared error, mean absolute error, root mean squared error, and R-squared score across all folds
print(f'Average mean squared error (5-fold cross-validation): {np.mean(mse_scores):.2f}')
print(f'Average mean absolute error (5-fold cross-validation): {np.mean(mae_scores):.2f}')
print(f'Average root mean squared error (5-fold cross-validation): {np.mean(rmse_scores):.2f}')
print(f'Average R-squared score (5-fold cross-validation): {np.mean(r2_scores):.2f}')


Cross Validation of using best parameters
Average mean squared error (5-fold cross-validation): 1.96
Average mean absolute error (5-fold cross-validation): 0.95
Average root mean squared error (5-fold cross-validation): 1.38
Average R-squared score (5-fold cross-validation): 0.37


##### GRU with Bayesain optimization and Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Global variable to store the best model
best_model = None
best_optimizer = None

# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(GRU(64, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(GRU(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

def objective_function(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    global best_optimizer

    # Round the optimizer index to the nearest integer
    optimizer_idx = round(optimizer_idx)

    model = create_model(optimizer_idx, learning_rate, dropout_rate, batch_size)

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    history = model.fit(X_train, y_train, epochs=100, batch_size=int(batch_size), validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Calculate the validation score
    y_val_pred = model.predict(X_val)
    val_score = r2_score(y_val, y_val_pred)

    # Update the best model if necessary
    if best_model is None or val_score > np.mean(r2_score(y_val, best_model.predict(X_val))):
        best_model = model
        best_optimizer = optimizers[optimizer_idx]

    # Return the validation score
    return val_score #last will be return

# Define the parameter space for Bayesian optimization
pbounds = {
    'optimizer_idx': (0, len(optimizers) - 1),
    'learning_rate': (0.0001, 0.1),
    'dropout_rate': (0.1, 0.5),
    'batch_size': (16, 64)
}

# Create the Bayesian optimization object
optimizer = BayesianOptimization(f=objective_function, pbounds=pbounds, random_state=7)

# Run the Bayesian optimization
# optimizer.maximize(init_points=5, n_iter=3)



In [ ]:
# Optimize the model
optimizer.maximize(init_points=5, n_iter=3)  # You can adjust the number of iterations

|   iter    |  target   | batch_... | dropou... | learni... | optimi... |
-------------------------------------------------------------------------
Epoch 1/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 3.3918 - r2_score: -0.0029 - val_loss: 8.0267 - val_r2_score: -1.4118
Epoch 2/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 2.0542 - r2_score: 0.3538 - val_loss: 2.8536 - val_r2_score: 0.1426
Epoch 3/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 1.8225 - r2_score: 0.4137 - val_loss: 1.5195 - val_r2_score: 0.5434
Epoch 4/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - loss: 1.6960 - r2_score: 0.4547 - val_loss: 3.6331 - val_r2_score: -0.0916
Epoch 5/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 1.6507 - r2_score: 0.4714 - val_loss: 2.1023 - val_r2_score: 0.3683
Epoch 6/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - loss: 1.6488 - r2_score: 0.4933 - val_loss: 4.8109 - val_r2_score: -0.4455
Epoch 7/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 1.6587

In [ ]:

# Print the best parameters found by Bayesian Optimization
print('Best parameters:', optimizer.max)
print('Best optimizer:', best_optimizer)

# Train the model with the best parameters
best_params = optimizer.max['params']
best_optimizer_idx = best_params['optimizer_idx']
best_learning_rate = best_params['learning_rate']
best_dropout_rate = best_params['dropout_rate']
best_batch_size = best_params['batch_size']

# Use the best model stored in the global variable
model = best_model

# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score using optimized model: {r2:.2f}')


# Store the average R-squared score
avg_score = optimizer.max['target']
print(f'Average R2 Score: {avg_score:.2f}')


Best parameters: {'target': 0.6899586617862221, 'params': {'batch_size': 46.13884820049921, 'dropout_rate': 0.3097745366784097, 'learning_rate': 0.029398488770955668, 'optimizer_idx': 1.547645986999483}}
Best optimizer: SGD
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Mean Squared Error: 1.05
Root Mean Squared Error: 1.03
Mean Absolute Error: 0.72
R2 Score using optimized model: 0.66
Average R2 Score: 0.69


GRU Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(GRU(64, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(GRU(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the number of folds
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

# Lists to store the scores for each fold
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []



In [ ]:
# Perform cross-validation
fold_number = 0
for train, val in kfold.split(X_train, y_train):
    fold_number += 1
    print(f"Starting fold {fold_number}")

    # Split the training data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train], X_train[val]

    # Use .iloc[] to access elements by integer position
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    # Create the model with the best parameters found by Bayesian optimization
    model = create_model(best_params['optimizer_idx'], best_params['learning_rate'], best_params['dropout_rate'], best_params['batch_size'])

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    # Train the model on the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=int(best_params['batch_size']), validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

    # Make predictions on the test data for this fold
    y_pred_fold = model.predict(X_test).flatten()

    # Calculate the mean squared error, mean absolute error, root mean squared error, and R-squared score for this fold
    mse = mean_squared_error(y_test, y_pred_fold)
    mae = mean_absolute_error(y_test, y_pred_fold)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_fold)

    # Append the scores to the lists of scores
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)


Starting fold 1
Epoch 1/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 3.2244 - r2_score: 0.0199 - val_loss: 2.0448 - val_r2_score: 0.3790
Epoch 2/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 1.6918 - r2_score: 0.4889 - val_loss: 1.6635 - val_r2_score: 0.4948
Epoch 3/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 1.4419 - r2_score: 0.5478 - val_loss: 1.5269 - val_r2_score: 0.5363
Epoch 4/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.2823 - r2_score: 0.5894 - val_loss: 1.1936 - val_r2_score: 0.6375
Epoch 5/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 1.1996 - r2_score: 0.6375 - val_loss: 1.2511 - val_r2_score: 0.6201
Epoch 6/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.9910 - r2_score: 0.6932 - val_loss: 1.1730 - val_r2_score: 0.6438
Epoch 7/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.0519 - r2_score: 0.6732 - val_loss: 1.0885 - val_r2_score: 0.6695
Epoch 8/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.9164 - r2_

In [ ]:

print("Cross Validation of using best parameters")
# Print the average mean squared error, mean absolute error, root mean squared error, and R-squared score across all folds
print(f'Average mean squared error (5-fold cross-validation): {np.mean(mse_scores):.2f}')
print(f'Average mean absolute error (5-fold cross-validation): {np.mean(mae_scores):.2f}')
print(f'Average root mean squared error (5-fold cross-validation): {np.mean(rmse_scores):.2f}')
print(f'Average R-squared score (5-fold cross-validation): {np.mean(r2_scores):.2f}')


Cross Validation of using best parameters
Average mean squared error (5-fold cross-validation): 1.15
Average mean absolute error (5-fold cross-validation): 0.76
Average root mean squared error (5-fold cross-validation): 1.07
Average R-squared score (5-fold cross-validation): 0.63


##### ConvLSTM1D with Bayesain optimization  and Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, BatchNormalization, Bidirectional, LSTM, ConvLSTM1D
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Global variable to store the best model
best_model = None
best_optimizer = None

# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2]))) # Change the input shape to reflect the reshaped data
    #The input data needs to be reshaped to be 4 dimensional
    model.add(tf.keras.layers.Reshape((1, 1, X_train.shape[2]))) # Reshape to (samples, time steps, rows, channels)
    model.add(ConvLSTM1D(64, kernel_size=1, return_sequences=True))  # Add first LSTM layer with return_sequences=True
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    # Remove the second reshape layer as it's not needed
    model.add(ConvLSTM1D(32, kernel_size=1))  # Add second LSTM layer with return_sequences=False
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(1))


    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

def objective_function(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    global best_optimizer

    # Round the optimizer index to the nearest integer
    optimizer_idx = round(optimizer_idx)

    model = create_model(optimizer_idx, learning_rate, dropout_rate, batch_size)

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    history = model.fit(X_train, y_train, epochs=100, batch_size=int(batch_size), validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Calculate the validation score
    y_val_pred = model.predict(X_val)
    val_score = r2_score(y_val, y_val_pred.flatten())

    # Update the best model if necessary
    if best_model is None or val_score > np.mean(r2_score(y_val, best_model.predict(X_val).flatten())):
        best_model = model
        best_optimizer = optimizers[optimizer_idx]

    # Return the validation score
    return val_score #last will be return

# Define the parameter space for Bayesian optimization
pbounds = {
    'optimizer_idx': (0, len(optimizers) - 1),
    'learning_rate': (0.0001, 0.1),
    'dropout_rate': (0.1, 0.5),
    'batch_size': (16, 64)
}

# Create the Bayesian optimization object
optimizer = BayesianOptimization(f=objective_function, pbounds=pbounds, random_state=7)

# Run the Bayesian optimization
# optimizer.maximize(init_points=5, n_iter=3)



In [ ]:
# Optimize the model
optimizer.maximize(init_points=5, n_iter=3)  # You can adjust the number of iterations

|   iter    |  target   | batch_... | dropou... | learni... | optimi... |
-------------------------------------------------------------------------
Epoch 1/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - loss: 3.2512 - r2_score: 0.0227 - val_loss: 3.6189 - val_r2_score: -0.0874
Epoch 2/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 2.4384 - r2_score: 0.2412 - val_loss: 3.4258 - val_r2_score: -0.0293
Epoch 3/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 2.4727 - r2_score: 0.2211 - val_loss: 2.9745 - val_r2_score: 0.1063
Epoch 4/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - loss: 2.5473 - r2_score: 0.1922 - val_loss: 2.2697 - val_r2_score: 0.3180
Epoch 5/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 2.6538 - r2_score: 0.1645 - val_loss: 3.0512 - val_r2_score: 0.0832
Epoch 6/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 2.6481 - r2_score: 0.1724 - val_loss: 3.4462 - val_r2_score: -0.0355
Epoch 7/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 2.6820

In [ ]:

# Print the best parameters found by Bayesian Optimization
print('Best parameters:', optimizer.max)
print('Best optimizer:', best_optimizer)

# Train the model with the best parameters
best_params = optimizer.max['params']
best_optimizer_idx = best_params['optimizer_idx']
best_learning_rate = best_params['learning_rate']
best_dropout_rate = best_params['dropout_rate']
best_batch_size = best_params['batch_size']

# Use the best model stored in the global variable
model = best_model

# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score using optimized model: {r2:.2f}')


# Store the average R-squared score
avg_score = optimizer.max['target']
print(f'Average R2 Score: {avg_score:.2f}')


Best parameters: {'target': 0.698319796438355, 'params': {'batch_size': 21.52661915935147, 'dropout_rate': 0.10830498558331786, 'learning_rate': 0.03951778462225156, 'optimizer_idx': 1.8220210364042873}}
Best optimizer: SGD
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Mean Squared Error: 0.99
Root Mean Squared Error: 0.99
Mean Absolute Error: 0.67
R2 Score using optimized model: 0.68
Average R2 Score: 0.70


ConvLSTM1D Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2]))) # Change the input shape to reflect the reshaped data
    #The input data needs to be reshaped to be 4 dimensional
    model.add(tf.keras.layers.Reshape((1, 1, X_train.shape[2]))) # Reshape to (samples, time steps, rows, channels)
    model.add(ConvLSTM1D(64, kernel_size=1, return_sequences=True))  # Add first LSTM layer with return_sequences=True
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    # Remove the second reshape layer as it's not needed
    model.add(ConvLSTM1D(32, kernel_size=1))  # Add second LSTM layer with return_sequences=False
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the number of folds
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

# Lists to store the scores for each fold
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []



In [ ]:
# Perform cross-validation
fold_number = 0
for train, val in kfold.split(X_train, y_train):
    fold_number += 1
    print(f"Starting fold {fold_number}")

    # Split the training data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train], X_train[val]

    # Use .iloc[] to access elements by integer position
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    # Create the model with the best parameters found by Bayesian optimization
    model = create_model(best_params['optimizer_idx'], best_params['learning_rate'], best_params['dropout_rate'], best_params['batch_size'])

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    # Train the model on the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=int(best_params['batch_size']), validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

    # Make predictions on the test data for this fold
    y_pred_fold = model.predict(X_test).flatten()

    # Calculate the mean squared error, mean absolute error, root mean squared error, and R-squared score for this fold
    mse = mean_squared_error(y_test, y_pred_fold)
    mae = mean_absolute_error(y_test, y_pred_fold)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_fold)

    # Append the scores to the lists of scores
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)


Starting fold 1
Epoch 1/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - loss: 3.1860 - r2_score: 0.0468 - val_loss: 2.0319 - val_r2_score: 0.3829
Epoch 2/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 2.0521 - r2_score: 0.3589 - val_loss: 1.9501 - val_r2_score: 0.4078
Epoch 3/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 1.9670 - r2_score: 0.3677 - val_loss: 1.9834 - val_r2_score: 0.3977
Epoch 4/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 1.9846 - r2_score: 0.3712 - val_loss: 2.1373 - val_r2_score: 0.3509
Epoch 5/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - loss: 2.0434 - r2_score: 0.3436 - val_loss: 1.9921 - val_r2_score: 0.3950
Epoch 6/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - loss: 2.0443 - r2_score: 0.3857 - val_loss: 1.8577 - val_r2_score: 0.4359
Epoch 7/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 1.9596 - r2_score: 0.3927 - val_loss: 2.0350 - val_r2_score: 0.3820
Epoch 8/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 2.0650 - r2

In [ ]:

print("Cross Validation of using best parameters")
# Print the average mean squared error, mean absolute error, root mean squared error, and R-squared score across all folds
print(f'Average mean squared error (5-fold cross-validation): {np.mean(mse_scores):.2f}')
print(f'Average mean absolute error (5-fold cross-validation): {np.mean(mae_scores):.2f}')
print(f'Average root mean squared error (5-fold cross-validation): {np.mean(rmse_scores):.2f}')
print(f'Average R-squared score (5-fold cross-validation): {np.mean(r2_scores):.2f}')


Cross Validation of using best parameters
Average mean squared error (5-fold cross-validation): 2.19
Average mean absolute error (5-fold cross-validation): 1.15
Average root mean squared error (5-fold cross-validation): 1.48
Average R-squared score (5-fold cross-validation): 0.30


##### BiConvLSTM1D with Bayesain optimization and Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, BatchNormalization, Bidirectional, LSTM, ConvLSTM1D
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Global variable to store the best model
best_model = None
best_optimizer = None

# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2]))) # Change the input shape to reflect the reshaped data
    #The input data needs to be reshaped to be 4 dimensional
    model.add(tf.keras.layers.Reshape((1, 1, X_train.shape[2]))) # Reshape to (samples, time steps, rows, channels)
    model.add(Bidirectional(ConvLSTM1D(64, kernel_size=1, return_sequences=True)))  # Add first LSTM layer with return_sequences=True
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    # Remove the second reshape layer as it's not needed
    model.add(Bidirectional(ConvLSTM1D(32, kernel_size=1)))  # Add second LSTM layer with return_sequences=False
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(1))


    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

def objective_function(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    global best_optimizer

    # Round the optimizer index to the nearest integer
    optimizer_idx = round(optimizer_idx)

    model = create_model(optimizer_idx, learning_rate, dropout_rate, batch_size)

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    history = model.fit(X_train, y_train, epochs=100, batch_size=int(batch_size), validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Calculate the validation score
    y_val_pred = model.predict(X_val)
    val_score = r2_score(y_val, y_val_pred.flatten())

    # Update the best model if necessary
    if best_model is None or val_score > np.mean(r2_score(y_val, best_model.predict(X_val).flatten())):
        best_model = model
        best_optimizer = optimizers[optimizer_idx]

    # Return the validation score
    return val_score #last will be return

# Define the parameter space for Bayesian optimization
pbounds = {
    'optimizer_idx': (0, len(optimizers) - 1),
    'learning_rate': (0.0001, 0.1),
    'dropout_rate': (0.1, 0.5),
    'batch_size': (16, 64)
}

# Create the Bayesian optimization object
optimizer = BayesianOptimization(f=objective_function, pbounds=pbounds, random_state=7)

# Run the Bayesian optimization
# optimizer.maximize(init_points=5, n_iter=3)



In [ ]:
# Optimize the model
optimizer.maximize(init_points=5, n_iter=3)  # You can adjust the number of iterations

|   iter    |  target   | batch_... | dropou... | learni... | optimi... |
-------------------------------------------------------------------------
Epoch 1/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 21s 29ms/step - loss: 3.6268 - r2_score: -0.0555 - val_loss: 2.6394 - val_r2_score: 0.2069
Epoch 2/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - loss: 2.5305 - r2_score: 0.1960 - val_loss: 2.2479 - val_r2_score: 0.3246
Epoch 3/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - loss: 2.5932 - r2_score: 0.1818 - val_loss: 3.8659 - val_r2_score: -0.1616
Epoch 4/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - loss: 2.5584 - r2_score: 0.1863 - val_loss: 5.2562 - val_r2_score: -0.5793
Epoch 5/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - loss: 2.9932 - r2_score: 0.0862 - val_loss: 2.9087 - val_r2_score: 0.1260
Epoch 6/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - loss: 2.7018 - r2_score: 0.1395 - val_loss: 2.3584 - val_r2_score: 0.2914
Epoch 7/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 2.8

In [ ]:

# Print the best parameters found by Bayesian Optimization
print('Best parameters:', optimizer.max)
print('Best optimizer:', best_optimizer)

# Train the model with the best parameters
best_params = optimizer.max['params']
best_optimizer_idx = best_params['optimizer_idx']
best_learning_rate = best_params['learning_rate']
best_dropout_rate = best_params['dropout_rate']
best_batch_size = best_params['batch_size']

# Use the best model stored in the global variable
model = best_model

# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score using optimized model: {r2:.2f}')


# Store the average R-squared score
avg_score = optimizer.max['target']
print(f'Average R2 Score: {avg_score:.2f}')


Best parameters: {'target': 0.7208893508014105, 'params': {'batch_size': 21.52661915935147, 'dropout_rate': 0.10830498558331786, 'learning_rate': 0.03951778462225156, 'optimizer_idx': 1.8220210364042873}}
Best optimizer: SGD
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Mean Squared Error: 0.94
Root Mean Squared Error: 0.97
Mean Absolute Error: 0.64
R2 Score using optimized model: 0.70
Average R2 Score: 0.72


BiConvLSTM1D Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2]))) # Change the input shape to reflect the reshaped data
    #The input data needs to be reshaped to be 4 dimensional
    model.add(tf.keras.layers.Reshape((1, 1, X_train.shape[2]))) # Reshape to (samples, time steps, rows, channels)
    model.add(Bidirectional(ConvLSTM1D(64, kernel_size=1, return_sequences=True)))  # Add first LSTM layer with return_sequences=True
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    # Remove the second reshape layer as it's not needed
    model.add(Bidirectional(ConvLSTM1D(32, kernel_size=1)))  # Add second LSTM layer with return_sequences=False
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the number of folds
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

# Lists to store the scores for each fold
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []



In [ ]:
# Perform cross-validation
fold_number = 0
for train, val in kfold.split(X_train, y_train):
    fold_number += 1
    print(f"Starting fold {fold_number}")

    # Split the training data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train], X_train[val]

    # Use .iloc[] to access elements by integer position
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    # Create the model with the best parameters found by Bayesian optimization
    model = create_model(best_params['optimizer_idx'], best_params['learning_rate'], best_params['dropout_rate'], best_params['batch_size'])

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    # Train the model on the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=int(best_params['batch_size']), validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

    # Make predictions on the test data for this fold
    y_pred_fold = model.predict(X_test).flatten()

    # Calculate the mean squared error, mean absolute error, root mean squared error, and R-squared score for this fold
    mse = mean_squared_error(y_test, y_pred_fold)
    mae = mean_absolute_error(y_test, y_pred_fold)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_fold)

    # Append the scores to the lists of scores
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)


Starting fold 1
Epoch 1/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - loss: 3.0742 - r2_score: 0.0426 - val_loss: 2.1407 - val_r2_score: 0.3499
Epoch 2/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - loss: 2.1427 - r2_score: 0.3271 - val_loss: 1.8672 - val_r2_score: 0.4330
Epoch 3/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.0718 - r2_score: 0.3432 - val_loss: 1.8579 - val_r2_score: 0.4358
Epoch 4/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - loss: 2.0397 - r2_score: 0.3559 - val_loss: 1.9603 - val_r2_score: 0.4047
Epoch 5/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 2.0842 - r2_score: 0.3278 - val_loss: 2.0280 - val_r2_score: 0.3841
Epoch 6/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.2185 - r2_score: 0.3065 - val_loss: 2.0499 - val_r2_score: 0.3775
Epoch 7/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - loss: 2.0401 - r2_score: 0.3598 - val_loss: 2.1711 - val_r2_score: 0.3407
Epoch 8/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - loss: 2.2140

In [ ]:

print("Cross Validation of using best parameters")
# Print the average mean squared error, mean absolute error, root mean squared error, and R-squared score across all folds
print(f'Average mean squared error (5-fold cross-validation): {np.mean(mse_scores):.2f}')
print(f'Average mean absolute error (5-fold cross-validation): {np.mean(mae_scores):.2f}')
print(f'Average root mean squared error (5-fold cross-validation): {np.mean(rmse_scores):.2f}')
print(f'Average R-squared score (5-fold cross-validation): {np.mean(r2_scores):.2f}')


Cross Validation of using best parameters
Average mean squared error (5-fold cross-validation): 2.49
Average mean absolute error (5-fold cross-validation): 1.25
Average root mean squared error (5-fold cross-validation): 1.57
Average R-squared score (5-fold cross-validation): 0.20


##### Conv1D Layer in Custom Model with Bayesain optimization  and Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, MaxPooling1D, GRU, Conv1D, Dense, Dropout, Flatten, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Global variable to store the best model
best_model = None
best_optimizer = None

# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    optimizer = optimizers[int(optimizer_idx)]

    model = Sequential()
    # No need to reshape here as X_train is already in the correct shape for Conv1D
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    # Remove the erroneous Reshape layer
    model.add(Conv1D(64, 3, activation='relu', padding='same'))
    # Adjust pool_size or remove MaxPooling1D if not needed
    model.add(MaxPooling1D(pool_size=1))  # Change pool_size to 1
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

def objective_function(optimizer_idx, learning_rate, dropout_rate, batch_size):
    global best_model
    global best_optimizer

    # Round the optimizer index to the nearest integer
    optimizer_idx = round(optimizer_idx)

    model = create_model(optimizer_idx, learning_rate, dropout_rate, batch_size)

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    history = model.fit(X_train, y_train, epochs=100, batch_size=int(batch_size), validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Calculate the validation score
    y_val_pred = model.predict(X_val)
    val_score = r2_score(y_val, y_val_pred)

    # Update the best model if necessary
    if best_model is None or val_score > np.mean(r2_score(y_val, best_model.predict(X_val))):
        best_model = model
        best_optimizer = optimizers[optimizer_idx]

    # Return the validation score
    return val_score #last will be return

# Define the parameter space for Bayesian optimization
pbounds = {
    'optimizer_idx': (0, len(optimizers) - 1),
    'learning_rate': (0.0001, 0.1),
    'dropout_rate': (0.1, 0.5),
    'batch_size': (16, 64)
}

# Create the Bayesian optimization object
optimizer = BayesianOptimization(f=objective_function, pbounds=pbounds, random_state=7)

# Run the Bayesian optimization
# optimizer.maximize(init_points=5, n_iter=3)



In [ ]:
# Optimize the model
optimizer.maximize(init_points=5, n_iter=3)  # You can adjust the number of iterations

|   iter    |  target   | batch_... | dropou... | learni... | optimi... |
-------------------------------------------------------------------------
Epoch 1/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 2.9612 - r2_score: 0.0777 - val_loss: 1.7442 - val_r2_score: 0.4759
Epoch 2/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 1.6776 - r2_score: 0.4610 - val_loss: 1.4388 - val_r2_score: 0.5677
Epoch 3/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 1.4138 - r2_score: 0.5529 - val_loss: 2.5278 - val_r2_score: 0.2405
Epoch 4/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 1.2693 - r2_score: 0.6026 - val_loss: 4.1583 - val_r2_score: -0.2494
Epoch 5/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 1.2341 - r2_score: 0.5979 - val_loss: 5.6435 - val_r2_score: -0.6957
Epoch 6/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 1.3075 - r2_score: 0.5915 - val_loss: 1.5191 - val_r2_score: 0.5436
Epoch 7/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 1.0995 - r2

In [ ]:

# Print the best parameters found by Bayesian Optimization
print('Best parameters:', optimizer.max)
print('Best optimizer:', best_optimizer)

# Train the model with the best parameters
best_params = optimizer.max['params']
best_optimizer_idx = best_params['optimizer_idx']
best_learning_rate = best_params['learning_rate']
best_dropout_rate = best_params['dropout_rate']
best_batch_size = best_params['batch_size']

# Use the best model stored in the global variable
model = best_model

# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {np.sqrt(mse):.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R2 Score using optimized model: {r2:.2f}')


# Store the average R-squared score
avg_score = optimizer.max['target']
print(f'Average R2 Score: {avg_score:.2f}')


Best parameters: {'target': 0.6985630845108926, 'params': {'batch_size': 28.885071044889816, 'dropout_rate': 0.299953000330224, 'learning_rate': 0.06795507661248196, 'optimizer_idx': 1.607478072208751}}
Best optimizer: SGD
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Mean Squared Error: 1.04
Root Mean Squared Error: 1.02
Mean Absolute Error: 0.72
R2 Score using optimized model: 0.67
Average R2 Score: 0.70


Conv1D Cross Validation

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# Set seed for reproducibility
np.random.seed(7)
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])



X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=7)

# Use the same function above for the validation set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

# Reshape the input data to have three dimensions
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Define a list of optimizers
optimizers = ['Adam', 'RMSprop', 'SGD']

def create_model(optimizer_idx, learning_rate, dropout_rate, batch_size):
    optimizer = optimizers[int(optimizer_idx)]

    # Initialize the model
    model = Sequential()
    # No need to reshape here as X_train is already in the correct shape for Conv1D
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    # Remove the erroneous Reshape layer
    model.add(Conv1D(64, 3, activation='relu', padding='same'))
    # Adjust pool_size or remove MaxPooling1D if not needed
    model.add(MaxPooling1D(pool_size=1))  # Change pool_size to 1
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
                  optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=learning_rate),
                  metrics=['r2_score'])

    return model # Return the compiled model

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the number of folds
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

# Lists to store the scores for each fold
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []



In [ ]:
# Perform cross-validation
fold_number = 0
for train, val in kfold.split(X_train, y_train):
    fold_number += 1
    print(f"Starting fold {fold_number}")

    # Split the training data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train], X_train[val]

    # Use .iloc[] to access elements by integer position
    y_train_fold, y_val_fold = y_train.iloc[train], y_train.iloc[val]

    # Create the model with the best parameters found by Bayesian optimization
    model = create_model(best_params['optimizer_idx'], best_params['learning_rate'], best_params['dropout_rate'], best_params['batch_size'])

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

    # Train the model on the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=int(best_params['batch_size']), validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

    # Make predictions on the test data for this fold
    y_pred_fold = model.predict(X_test).flatten()

    # Calculate the mean squared error, mean absolute error, root mean squared error, and R-squared score for this fold
    mse = mean_squared_error(y_test, y_pred_fold)
    mae = mean_absolute_error(y_test, y_pred_fold)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_fold)

    # Append the scores to the lists of scores
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)


Starting fold 1
Epoch 1/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 3.3566 - r2_score: -0.0240 - val_loss: 5.1705 - val_r2_score: -0.5702
Epoch 2/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 1.7093 - r2_score: 0.4561 - val_loss: 5.2060 - val_r2_score: -0.5810
Epoch 3/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 1.4838 - r2_score: 0.5171 - val_loss: 2.0135 - val_r2_score: 0.3885
Epoch 4/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 1.2900 - r2_score: 0.5969 - val_loss: 1.8728 - val_r2_score: 0.4312
Epoch 5/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.3194 - r2_score: 0.5888 - val_loss: 1.2264 - val_r2_score: 0.6276
Epoch 6/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.1501 - r2_score: 0.6354 - val_loss: 1.2316 - val_r2_score: 0.6260
Epoch 7/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.1615 - r2_score: 0.6433 - val_loss: 1.3331 - val_r2_score: 0.5951
Epoch 8/100
208/208 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 1.2032 - r2_sco

In [ ]:

print("Cross Validation of using best parameters")
# Print the average mean squared error, mean absolute error, root mean squared error, and R-squared score across all folds
print(f'Average mean squared error (5-fold cross-validation): {np.mean(mse_scores):.2f}')
print(f'Average mean absolute error (5-fold cross-validation): {np.mean(mae_scores):.2f}')
print(f'Average root mean squared error (5-fold cross-validation): {np.mean(rmse_scores):.2f}')
print(f'Average R-squared score (5-fold cross-validation): {np.mean(r2_scores):.2f}')


Cross Validation of using best parameters
Average mean squared error (5-fold cross-validation): 1.45
Average mean absolute error (5-fold cross-validation): 0.88
Average root mean squared error (5-fold cross-validation): 1.20
Average R-squared score (5-fold cross-validation): 0.54


##### Deployment

In [86]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [87]:
# Load the dataset
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')

epsilon = 1e-12

# Handle zero and negative values in IC50_M
df['pIC50'] = df['standard_value'].replace(0, np.nan) / 1e6
df['pIC50'] = -np.log10(df['pIC50'])
df['pIC50'] = df['pIC50'].apply(lambda x: x if x > 0 else epsilon)


# Remove rows with missing values
df = df.dropna(subset=['pIC50'])

# Handle potential missing or non-string values in 'canonical_smiles'
df['canonical_smiles'] = df['canonical_smiles'].fillna('') # Replace missing values with empty strings
df['canonical_smiles'] = df['canonical_smiles'].astype(str) # Ensure all values are strings

# Generate compound descriptors using RDKit
mols = [Chem.MolFromSmiles(smiles) for smiles in df['canonical_smiles']]

# Using MorganGenerator
morgan_generator = GetMorganGenerator(radius=2)
fps = [morgan_generator.GetFingerprint(mol) for mol in mols]

# Determine the size of the fingerprint
n_bits = fps[0].GetNumBits()

# Convert fingerprints to numpy arrays
X = np.zeros((len(fps), n_bits))
for i, fp in enumerate(fps):
    DataStructs.ConvertToNumpyArray(fp, X[i])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['pIC50'], test_size=0.2, random_state=42)

In [88]:
# Train a random forest regressor
rf = XGBRegressor(n_estimators=500)
rf.fit(X_train, y_train)
rf

# Evaluate the model
# y_pred = rf.predict(X_test)
# print('R-squared:', rf.score(X_test, y_test))

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [89]:
# saving model
import pickle
with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

In [90]:
# Loading model
with open('xgb_model.pkl', 'rb') as f:
    Xgb_model = pickle.load(f)

User Input

In [106]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator

# Enter the SMILES string
smiles = input("Enter the SMILES string: ")

# Generate a molecule object from the SMILES string
mol = Chem.MolFromSmiles(smiles)

if mol is not None:
    # Using MorganGenerator
    morgan_generator = GetMorganGenerator(radius=2)
    fp = morgan_generator.GetFingerprint(mol)

    # Determine the size of the fingerprint
    n_bits = fp.GetNumBits()

    # Convert fingerprints to numpy arrays
    X = np.zeros((1, n_bits))
    DataStructs.ConvertToNumpyArray(fp, X[0])

    # Print the preprocessed data
    print("Preprocessed data:", X)
else:
    print("Could not generate a valid molecule from the provided SMILES string.")

# predict
smile_string = Xgb_model.predict(X) # Changed xgb_model to Xgb_model
print(smile_string)

if smile_string >= 8:
    print("The molecule is active and considered a good candidate for drug development, as the predicted pIC50 value is greater than or equal to 8.")
else:
    print("The molecule is inactive and is not considered a good candidate for drug development, as the predicted pIC50 value is less than 8.")

Enter the SMILES string: CC(C)=CC(=O)N[C@H]1CC[C@@]2(C)[C@@H](CC[C@@H]3[C@@H]2CC[C@]2(C)C([C@H](C)N(C)C)=CC[C@@H]32)[C@H]1O
Preprocessed data: [[0. 1. 0. ... 0. 0. 0.]]
[1.4426998]
The molecule is inactive and is not considered a good candidate for drug development, as the predicted pIC50 value is less than 8.


In [108]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [124]:
%%writefile app.py

import streamlit as st
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
import pickle

# Load the trained model
with open('xgb_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Set the app title
st.title('Molecule Activity Prediction')

# Create a text input for the user to enter a SMILES string
smiles = st.text_input('Enter a SMILES string')

# If the user has entered a SMILES string, preprocess it and make a prediction
if smiles:
    # Generate a molecule object from the SMILES string
    mol = Chem.MolFromSmiles(smiles)

    if mol is not None:
        # Using MorganGenerator
        morgan_generator = GetMorganGenerator(radius=2)
        fp = morgan_generator.GetFingerprint(mol)

        # Determine the size of the fingerprint
        n_bits = fp.GetNumBits()

        # Convert fingerprints to numpy arrays
        X = np.zeros((1, n_bits))
        DataStructs.ConvertToNumpyArray(fp, X[0])

        # Make a prediction using the trained model
        pIC50 = model.predict(X)[0]

        # Display the prediction to the user
        if pIC50 >= 8:
            st.write('The molecule is active and considered a good candidate for drug development, as the predicted pIC50 value is greater than or equal to 8.')
            st.write(f'Predicted pIC50 value: {pIC50:.2f}')
        else:
            st.write('The molecule is inactive and is not considered a good candidate for drug development, as the predicted pIC50 value is less than 8.')
            st.write(f'Predicted pIC50 value: {pIC50:.2f}')
    else:
        st.write('Could not generate a valid molecule from the provided SMILES string.')


Overwriting app.py


In [113]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [115]:
!npm audit fix


up to date, audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

# npm audit report

axios  0.8.1 - 0.27.2
Severity: moderate
Axios Cross-Site Request Forgery Vulnerability - https://github.com/advisories/GHSA-wf5p-g6vw-rhxx
fix available via `npm audit fix --force`
Will install localtunnel@1.8.3, which is a breaking change
node_modules/axios
  localtunnel  >=1.9.0
  Depends on vulnerable versions of axios
  node_modules/localtunnel

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force


In [125]:
!streamlit run app.py &>/content/logs.txt &

In [126]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.245.230.59


In [127]:
!npx localtunnel --port 8501

your url is: https://dry-symbols-fetch.loca.lt
^C


In [111]:
# !streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py --server.port=5050





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:5050
  Network URL: http://172.28.0.12:5050
  External URL: http://35.245.230.59:5050

  Stopping...
  Stopping...
